In [2]:
import fastf1
fastf1.Cache.enable_cache('../data/cache') 

In [230]:
# Set some variables
DRIVER = 'ALO'
DRIVER_NUMBER = 14
YEAR = 2022
RACE = 'Austria'

In [4]:
session = fastf1.get_session(YEAR, RACE, 'Race')
session.load(telemetry=True, laps=True, weather=False)

core           INFO 	Loading data for Austrian Grand Prix - Race [v2.2.9]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '44', '63', '31', '47', '4', '20', '3', '14', '77', '23', '18', '24', '10', '22', '5', '55', '6', '11']


In [190]:
# Find race date
print(session.event)
for i in range(1, 6):
    sn = getattr(session.event, f'Session{i}')
    if sn == 'Race':
        date = getattr(session.event, f'Session{i}Date').date()
        break
print(date)

RoundNumber                                                         11
Country                                                        Austria
Location                                                     Spielberg
OfficialEventName    FORMULA 1 ROLEX GROSSER PREIS VON ÖSTERREICH 2022
EventDate                                          2022-07-10 17:00:00
EventName                                          Austrian Grand Prix
EventFormat                                                     sprint
Session1                                                    Practice 1
Session1Date                                       2022-07-08 13:30:00
Session2                                                    Qualifying
Session2Date                                       2022-07-08 17:00:00
Session3                                                    Practice 2
Session3Date                                       2022-07-09 12:30:00
Session4                                                        Sprint
Sessio

In [5]:
from IPython.display import display, Markdown, HTML
import pandas

In [229]:
# Fastest Lap Time
laps = session.laps.pick_driver(DRIVER)
fastest_lap = laps.pick_fastest()

display(Markdown('### Fastest Lap Time'))
print(fastest_lap["LapTime"].to_pytimedelta())

### Fastest Lap Time

0:01:11.843000


In [32]:
# Average Lap Time
# Excludes when pit stop occurred
laps = session.laps.pick_driver(DRIVER)
average = laps.query('PitOutTime == "" and PitInTime == ""').filter(items=['LapTime']).mean()['LapTime']

display(Markdown('### Average Lap Time'))
print(average.round('ms').to_pytimedelta())

### Average Lap Time

0:01:09.983000


In [211]:
# Pit Stop Times
last_pit_in = None
pit_stops = []

laps = session.laps.pick_driver(DRIVER)
_laps = laps.filter(items=['LapNumber', 'PitInTime', 'PitOutTime'])
for idx, lap in _laps.iterrows():
    lap_number = lap['LapNumber']
    pit_in = lap['PitInTime']
    pit_out = lap['PitOutTime']
    
    if last_pit_in and not pandas.isnull(pit_out):
        pit_stops.append((lap_number-1, pandas.Timedelta(pit_out - last_pit_in)))
        last_pit_in = None
    if not pandas.isnull(pit_in):
        last_pit_in = pit_in
    
df = pandas.DataFrame(data=pit_stops, columns=['lap', 'time'])
average = df.filter(items=['time']).mean()['time'].round('ms').to_pytimedelta()
total = df.filter(items=['time']).sum()['time'].round('ms').to_pytimedelta()

display(Markdown('### Pit Stop Times'))
display(Markdown('#### Average'))
print(average)
display(Markdown('#### Total'))
print(total)
# For some reason these are slightly off from ergast pit information API.
# We consider pit time here as difference between when car entered the pit and exited.
# Maybe the API treats "pit duration" sligtly differently?
display(Markdown('#### By Lap'))
for lap, time in pit_stops:
    print(f'Lap {lap} - {time.round("ms").to_pytimedelta()}')


### Pit Stop Times

#### Average

0:00:23.674000


#### Total

0:00:23.674000


#### By Lap

Lap 1 - 0:00:23.674000


In [215]:
laps = session.laps.pick_driver(DRIVER)
fastest_lap = laps.pick_fastest()

# Average Speed
display(Markdown('### Average Speed'))
display(Markdown('#### Fastest Lap'))

car_data = fastest_lap.get_car_data()
print(car_data.mean(numeric_only=True)['Speed'].round(3))

display(Markdown('#### All Laps'))
# Excludes pit stop
car_data = laps.query('PitOutTime == "" and PitInTime == ""').get_car_data()
print(car_data.mean(numeric_only=True)['Speed'].round(3))

### Average Speed

#### Fastest Lap

229.084


#### All Laps

211.194


In [10]:
laps = session.laps.pick_driver(DRIVER)
fastest_lap = laps.pick_fastest()

# Speed Traps
display(Markdown('### Speed Traps'))

display(Markdown('#### Fastest Lap'))
display(Markdown('##### Intermediate 1'))
print(fastest_lap["SpeedI1"])
display(Markdown('##### Intermediate 2'))
print(fastest_lap["SpeedI2"])
display(Markdown('##### Speed Trap'))
print(fastest_lap["SpeedST"])
display(Markdown('##### Finising Line'))
print(fastest_lap["SpeedFL"])

display(Markdown('#### All Laps'))
df = laps.filter(items=['SpeedI1', 'SpeedI2', 'SpeedST', 'SpeedFL']).max()
display(Markdown('##### Intermediate 1'))
print(df["SpeedI1"])
display(Markdown('##### Intermediate 2'))
print(df["SpeedI2"])
display(Markdown('##### Speed Trap'))
print(df["SpeedST"])
display(Markdown('##### Finising Line'))
print(df["SpeedFL"])

### Speed Traps

#### Fastest Lap

##### Intermediate 1

305.0


##### Intermediate 2

...

238.0


##### Speed Trap

321.0


##### Finising Line

281.0


In [11]:
laps = session.laps.pick_driver(DRIVER)
fastest_lap = laps.pick_fastest()

display(Markdown('### Sector Times'))

# Sector Average
df_avg = laps.filter(items=['Sector1Time', 'Sector2Time', 'Sector3Time']).mean().round('ms')
display(Markdown('#### Average'))
display(df_avg)

# Sector Fastest
df_max = laps.filter(items=['Sector1Time', 'Sector2Time', 'Sector3Time']).min().round('ms')
display(Markdown('#### Fastest'))
display(df_max)

### Sector Times

#### Average

Sector1Time   0 days 00:00:18.396000
Sector2Time   0 days 00:00:31.474000
Sector3Time   0 days 00:00:21.477000
dtype: timedelta64[ns]

#### Fastest

Sector1Time   0 days 00:00:17.086000
Sector2Time   0 days 00:00:29.866000
Sector3Time   0 days 00:00:20.265000
dtype: timedelta64[ns]

In [272]:
this_driver = DRIVER_NUMBER
driver_ahead = None
threashold = pandas.Timedelta(3, 's')
pit_in_threashold = pandas.Timedelta(3, 's')
pit_out_threashold = pandas.Timedelta(3, 's')
car_length_threashold = 6
laps = session.laps.pick_driver(DRIVER)
max_laps = session.laps["LapNumber"].max()
print(max_laps)

driver_tel = session.laps.pick_driver(this_driver).get_car_data()

for i, lap in laps.iterlaps():
    drivers_already_passed = []
    lap_number = lap["LapNumber"]
    if lap_number == 1: # exclude first lap
        continue
        
    # Consider successful overtake as:
    # - driver ahead has changed
    # - driver maintains the position until finish line
    # - passed driver has not been lapped
    # - passed driver is not pitted
    # - passed driver is not off track
    # Consider adding checks for:
    # - track status (no yellow flags, sc leading to lapped cars making their way around).
    
    # Currently, if a car is side by side and the driver is slightly in front, gets away for a split second counts as an overtake.
    lap_telemetry = lap.get_car_data().add_driver_ahead()
    for _, t in lap_telemetry.iterrows():
        if driver_ahead == t['DriverAhead']:
            continue
 
        maybe_passed_driver = driver_ahead
        driver_ahead = t['DriverAhead']
        
        if not maybe_passed_driver:
            continue
            
        if maybe_passed_driver in drivers_already_passed:
            continue
            
        t0 = t['SessionTime']

        # Passed driver data
        pd_laps = session.laps.pick_driver(maybe_passed_driver)
        pd_lap_idx = pd_laps[(pd_laps['LapStartTime'] <= t0)]['LapStartTime'].idxmax()
        pd_lap = session.laps.iloc[pd_lap_idx]
        pd_pos = pd_laps.get_pos_data().slice_by_time(t0 - threashold, t0 + threashold)
        pd_tel = pd_laps.get_car_data()
        try:
            pd_tel_before = pd_tel[(pd_tel['SessionTime'] >= (
                t0 - threashold)) & (pd_tel['SessionTime'] < t0)].add_driver_ahead()
            pd_tel_after = pd_tel[(pd_tel['SessionTime'] >= t0) & (
                pd_tel['SessionTime'] < (t0 + threashold))].add_driver_ahead().add_distance()
        except (ValueError, IndexError):
            continue
        
        # Driver data
        try:
            driver_tel_after = driver_tel[(driver_tel['SessionTime'] >= t0) & (
                driver_tel['SessionTime'] < (t0 + threashold))].add_driver_ahead().add_distance()
        except ValueError:
            continue
            
        # Ensure the car is not off track
        if not pd_pos.query(f'Status == "OffTrack"').empty:
            continue
            
        # Check passed driver has not been lapped
        if pd_lap['LapNumber'] < lap_number:
            continue
        
        # Check for possible overtake based on driver ahead during the sample windows
        if not(not pd_tel_before.query(f'DriverAhead == "{driver_ahead}"').empty and not pd_tel_after.query(f'DriverAhead == "{this_driver}"').empty):
            continue

        # The drive is at least 1+ car length ahead at any point in time during t0 + 3 seconds
        if pd_tel_after.query(f'DriverAhead == "{this_driver}" and DistanceToDriverAhead >= {car_length_threashold}').empty:
            continue 
        
        # Check the driver has driven longer distance since t0
        if not driver_tel_after.iloc[-1]['Distance'] > pd_tel_after.iloc[-1]['Distance']:
            continue
        
        # Check passed driver was not pitted
        # This is determined true if t0 does not occur during any PitInTime - 3 second and PitOutTime + 3 second
        # TODO: this could be optimized to filter laps we only care about
        pit_stops = []
        was_pitted = False
        last_pit_in = None
        _pd_laps = pd_laps[(pd_laps["LapNumber"] >= lap_number - 1) & (pd_laps["LapNumber"] <= lap_number + 1)]
        for _, _pd_lap in _pd_laps.iterrows():
            pit_in = _pd_lap['PitInTime']
            pit_out = _pd_lap['PitOutTime']
            
            if not pandas.isnull(pit_in):
                if t0 <= (pit_in + pit_in_threashold) and t0 >= (pit_in - pit_in_threashold):
                    was_pitted = True
                    break
                last_pit_in = pit_in
            
            if not pandas.isnull(pit_out):
                if t0 <= (pit_out + pit_out_threashold) and t0 >= (pit_in - pit_out_threashold):
                    was_pitted = True
                    break
                if last_pit_in:
                    if t0 >= (last_pit_in - pit_in_threashold) and t0 <= (pit_out + pit_out_threashold):
                        was_pitted = True
                        break
                    last_pit_in = None
        if was_pitted:
            continue
            
        # Ensure the position was maintained to the finish line
        if not lap_number < max_laps:
            continue
        driver_lap_finished = lap["LapStartTime"] + lap["LapTime"]
        pd_lap_finished = pd_lap["LapStartTime"] + pd_lap["LapTime"]
        if driver_lap_finished > pd_lap_finished:
            continue
        
        drivers_already_passed.append(maybe_passed_driver)
        
        display()
        display(_pd_laps)
        display(_pd_laps.get_pos_data())
        display(pandas.unique(_pd_laps.get_pos_data()["Status"]))
        display(laps[(laps["LapNumber"] >= lap_number - 1) & (laps["LapNumber"] <= lap_number + 1)])
        display(t)

        print()
        print(f'--- Lap {lap_number} ---')
        print(f'-- Overtake --')
        print(f'Driver ahead before: {maybe_passed_driver}')
        print(f'Driver ahead now: {driver_ahead}')

71


,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,IsPersonalBest,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate
1125,0 days 01:03:35.615000,5,NaT,1,1,0 days 00:22:51.592000,NaT,NaT,0 days 00:00:34.643000,0 days 00:00:23.094000,...,False,HARD,2.0,False,0 days 01:02:13.798000,Aston Martin,VET,1,False,2022-07-10 13:03:13.807
1126,0 days 01:04:49.899000,5,0 days 00:01:14.284000,2,1,NaT,NaT,0 days 00:00:18.392000,0 days 00:00:33.068000,0 days 00:00:22.824000,...,False,HARD,3.0,False,0 days 01:03:35.615000,Aston Martin,VET,1,True,2022-07-10 13:04:35.624
1127,0 days 01:06:02.452000,5,0 days 00:01:12.553000,3,1,NaT,NaT,0 days 00:00:17.781000,0 days 00:00:32.377000,0 days 00:00:22.395000,...,False,HARD,4.0,False,0 days 01:04:49.899000,Aston Martin,VET,1,True,2022-07-10 13:05:49.908


,Date,Status,X,Y,Z,Source,Time,SessionTime
0,2022-07-10 13:03:13.867,OnTrack,1459,-1112,7215,pos,0 days 00:00:00.060000,0 days 01:02:13.858000
1,2022-07-10 13:03:14.047,OnTrack,1459,-1112,7215,pos,0 days 00:00:00.240000,0 days 01:02:14.038000
2,2022-07-10 13:03:14.427,OnTrack,1457,-1112,7215,pos,0 days 00:00:00.620000,0 days 01:02:14.418000
3,2022-07-10 13:03:14.627,OnTrack,1451,-1114,7215,pos,0 days 00:00:00.820000,0 days 01:02:14.618000
4,2022-07-10 13:03:14.807,OnTrack,1434,-1119,7215,pos,0 days 00:00:01,0 days 01:02:14.798000
...,...,...,...,...,...,...,...,...
867,2022-07-10 13:07:01.487,OnTrack,1890,-995,7220,pos,0 days 00:03:47.680000,0 days 01:06:01.478000
868,2022-07-10 13:07:01.827,OnTrack,1649,-1060,7217,pos,0 days 00:03:48.020000,0 days 01:06:01.818000
869,2022-07-10 13:07:02.007,OnTrack,1496,-1102,7215,pos,0 days 00:03:48.200000,0 days 01:06:01.998000
870,2022-07-10 13:07:02.247,OnTrack,1320,-1149,7215,pos,0 days 00:03:48.440000,0 days 01:06:02.238000


array(['OnTrack'], dtype=object)

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,IsPersonalBest,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate
635,0 days 01:03:35.917000,14,NaT,1,1,0 days 00:23:26.856000,NaT,NaT,0 days 00:00:34.712000,0 days 00:00:23.229000,...,False,HARD,1.0,True,0 days 01:02:13.798000,Alpine,ALO,1,False,2022-07-10 13:03:13.807
636,0 days 01:04:49.484000,14,0 days 00:01:13.567000,2,1,NaT,NaT,0 days 00:00:18.045000,0 days 00:00:33.148000,0 days 00:00:22.374000,...,False,HARD,2.0,True,0 days 01:03:35.917000,Alpine,ALO,1,True,2022-07-10 13:04:35.926
637,0 days 01:06:01.589000,14,0 days 00:01:12.105000,3,1,NaT,NaT,0 days 00:00:17.676000,0 days 00:00:32.408000,0 days 00:00:22.021000,...,False,HARD,3.0,True,0 days 01:04:49.484000,Alpine,ALO,1,True,2022-07-10 13:05:49.493


...

array(['OnTrack'], dtype=object)

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,...,IsPersonalBest,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate
645,0 days 01:15:39.059000,14,0 days 00:01:12.413000,11,1,NaT,NaT,0 days 00:00:17.689000,0 days 00:00:32.336000,0 days 00:00:22.388000,...,False,HARD,11.0,True,0 days 01:14:26.646000,Alpine,ALO,1,True,2022-07-10 13:15:26.655
646,0 days 01:16:51.841000,14,0 days 00:01:12.782000,12,1,NaT,NaT,0 days 00:00:17.578000,0 days 00:00:32.399000,0 days 00:00:22.805000,...,False,HARD,12.0,True,0 days 01:15:39.059000,Alpine,ALO,1,True,2022-07-10 13:16:39.068
647,0 days 01:18:04.285000,14,0 days 00:01:12.444000,13,1,NaT,NaT,0 days 00:00:18.089000,0 days 00:00:32.437000,0 days 00:00:21.918000,...,False,HARD,13.0,True,0 days 01:16:51.841000,Alpine,ALO,1,True,2022-07-10 13:17:51.850


Date                     2022-07-10 13:17:43.468000
RPM                                           11459
Speed                                           192
nGear                                             4
Throttle                                        100
Brake                                         False
DRS                                               0
Source                                          car
Time                         0 days 00:01:04.400000
SessionTime                  0 days 01:16:43.459000
DriverAhead                                      24
DistanceToDriverAhead                    100.764722
Name: 242, dtype: object


--- Lap 12 ---
-- Overtake --
Driver ahead before: 6
Driver ahead now: 24


KeyboardInterrupt: 

In [276]:
laps = session.laps.pick_driver(DRIVER)
lap_number = 3
laps = laps[(laps["LapNumber"] >= lap_number - 1) & (laps["LapNumber"] <= lap_number + 1)]
print(laps)

for i, lap in laps.iterlaps():
    display(lap.get_pos_data())


                      Time DriverNumber                LapTime  LapNumber  \
636 0 days 01:04:49.484000           14 0 days 00:01:13.567000          2   
637 0 days 01:06:01.589000           14 0 days 00:01:12.105000          3   
638 0 days 01:07:13.766000           14 0 days 00:01:12.177000          4   

     Stint PitOutTime PitInTime            Sector1Time            Sector2Time  \
636      1        NaT       NaT 0 days 00:00:18.045000 0 days 00:00:33.148000   
637      1        NaT       NaT 0 days 00:00:17.676000 0 days 00:00:32.408000   
638      1        NaT       NaT 0 days 00:00:17.440000 0 days 00:00:32.499000   

               Sector3Time  ... IsPersonalBest Compound TyreLife  FreshTyre  \
636 0 days 00:00:22.374000  ...          False     HARD      2.0       True   
637 0 days 00:00:22.021000  ...          False     HARD      3.0       True   
638 0 days 00:00:22.238000  ...          False     HARD      4.0       True   

              LapStartTime    Team  Driver  Track

,Date,Status,X,Y,Z,Source,Time,SessionTime
0,2022-07-10 13:04:36.107,OnTrack,1008,-1234,7215,pos,0 days 00:00:00.181000,0 days 01:03:36.098000
1,2022-07-10 13:04:36.407,OnTrack,740,-1306,7214,pos,0 days 00:00:00.481000,0 days 01:03:36.398000
2,2022-07-10 13:04:36.847,OnTrack,350,-1411,7214,pos,0 days 00:00:00.921000,0 days 01:03:36.838000
3,2022-07-10 13:04:37.107,OnTrack,259,-1436,7214,pos,0 days 00:00:01.181000,0 days 01:03:37.098000
4,2022-07-10 13:04:37.307,OnTrack,107,-1477,7214,pos,0 days 00:00:01.381000,0 days 01:03:37.298000
...,...,...,...,...,...,...,...,...
280,2022-07-10 13:05:48.388,OnTrack,1976,-972,7222,pos,0 days 00:01:12.462000,0 days 01:04:48.379000
281,2022-07-10 13:05:48.627,OnTrack,1808,-1018,7219,pos,0 days 00:01:12.701000,0 days 01:04:48.618000
282,2022-07-10 13:05:48.907,OnTrack,1609,-1071,7216,pos,0 days 00:01:12.981000,0 days 01:04:48.898000
283,2022-07-10 13:05:49.067,OnTrack,1494,-1102,7215,pos,0 days 00:01:13.141000,0 days 01:04:49.058000


,Date,Status,X,Y,Z,Source,Time,SessionTime
0,2022-07-10 13:05:49.527,OnTrack,999,-1236,7215,pos,0 days 00:00:00.034000,0 days 01:04:49.518000
1,2022-07-10 13:05:49.907,OnTrack,819,-1285,7214,pos,0 days 00:00:00.414000,0 days 01:04:49.898000
2,2022-07-10 13:05:50.167,OnTrack,562,-1354,7214,pos,0 days 00:00:00.674000,0 days 01:04:50.158000
3,2022-07-10 13:05:50.467,OnTrack,394,-1399,7214,pos,0 days 00:00:00.974000,0 days 01:04:50.458000
4,2022-07-10 13:05:50.807,OnTrack,54,-1491,7214,pos,0 days 00:00:01.314000,0 days 01:04:50.798000
...,...,...,...,...,...,...,...,...
265,2022-07-10 13:07:00.528,OnTrack,1920,-987,7221,pos,0 days 00:01:11.035000,0 days 01:06:00.519000
266,2022-07-10 13:07:00.768,OnTrack,1748,-1034,7218,pos,0 days 00:01:11.275000,0 days 01:06:00.759000
267,2022-07-10 13:07:00.928,OnTrack,1631,-1065,7217,pos,0 days 00:01:11.435000,0 days 01:06:00.919000
268,2022-07-10 13:07:01.227,OnTrack,1408,-1125,7215,pos,0 days 00:01:11.734000,0 days 01:06:01.218000


,Date,Status,X,Y,Z,Source,Time,SessionTime
0,2022-07-10 13:07:01.827,OnTrack,966,-1245,7214,pos,0 days 00:00:00.229000,0 days 01:06:01.818000
1,2022-07-10 13:07:02.007,OnTrack,826,-1283,7214,pos,0 days 00:00:00.409000,0 days 01:06:01.998000
2,2022-07-10 13:07:02.247,OnTrack,638,-1333,7214,pos,0 days 00:00:00.649000,0 days 01:06:02.238000
3,2022-07-10 13:07:02.407,OnTrack,511,-1368,7214,pos,0 days 00:00:00.809000,0 days 01:06:02.398000
4,2022-07-10 13:07:02.707,OnTrack,192,-1454,7214,pos,0 days 00:00:01.109000,0 days 01:06:02.698000
...,...,...,...,...,...,...,...,...
276,2022-07-10 13:08:12.688,OnTrack,1843,-1008,7220,pos,0 days 00:01:11.090000,0 days 01:07:12.679000
277,2022-07-10 13:08:13.088,OnTrack,1615,-1070,7216,pos,0 days 00:01:11.490000,0 days 01:07:13.079000
278,2022-07-10 13:08:13.248,OnTrack,1498,-1101,7215,pos,0 days 00:01:11.650000,0 days 01:07:13.239000
279,2022-07-10 13:08:13.428,OnTrack,1366,-1137,7215,pos,0 days 00:01:11.830000,0 days 01:07:13.419000
